In [13]:
import numpy as np
from glob import glob

In [14]:
from bcg import load_bcg
from fs import load_ecg
import plotly.graph_objs as go
from plotly.offline import plot

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
%reload_ext autoreload

In [17]:
data_dir = "/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/"

In [18]:
bcg_files = glob(data_dir+"*.txt", recursive=False)
ecg_files = glob(data_dir+"*.edf", recursive=False)

In [19]:
bcg_files.sort()
ecg_files.sort()

In [7]:
import autopeaks

In [8]:
def find_peaks_bcg(signal):
    bcg_auto_peaks = autopeaks.AutoPeaks(thres=0.73,min_dist=300,buffer_size=1000)
    list(map(bcg_auto_peaks.findpeaks, np.diff(signal)))
    return bcg_auto_peaks.peak_indexes, bcg_auto_peaks.peak_values

def find_peaks_ecg(signal):
    ecg_auto_peaks = autopeaks.AutoPeaks(thres=0.7,min_dist=300,buffer_size=1000)
    list(map(ecg_auto_peaks.findpeaks, signal))
    return ecg_auto_peaks.peak_indexes, ecg_auto_peaks.peak_values


In [50]:
import time

def mean_interval(intervals):
    hist,bin_edges = np.histogram(intervals,bins=5,range=(300,750))
    ind = np.argmax(hist)
    min, max = bin_edges[ind], bin_edges[ind+1]
    cluster = []
    for interval in intervals:
        if interval >= min and interval <= max:
            cluster.append(interval)
    return np.median(cluster)
    
for bcg_file,ecg_file in zip(bcg_files, ecg_files):
    if "yjj" not in bcg_file:
        continue
    print(bcg_file)
    bcg = load_bcg(bcg_file, filter=True, notch=False,offset=39558-396 )
    ecg = load_ecg(ecg_file, filter=True)
    bcg = -np.asarray(bcg)
    bcg_peak_indices, bcg_peak_values = find_peaks_bcg(bcg)
    ecg_peak_indices, ecg_peak_values = find_peaks_ecg(ecg)
    if False:
        plot([go.Scatter(y=ecg),go.Scatter(x=ecg_peak_indices,y=ecg_peak_values,mode="markers")])
        time.sleep(2)
        plot([go.Scatter(y=bcg),go.Scatter(x=bcg_peak_indices,y=bcg_peak_values,mode="markers")])
        time.sleep(2)
    bcg_intervals = np.diff(bcg_peak_indices)
    mean_bcg_intervals = []
    for ind in range(len(bcg_intervals)):
        mean_bcg_intervals.append(mean_interval(bcg_intervals[ind-15 if ind-10>=0 else 0:ind+10]))
    ecg_intervals = np.diff(ecg_peak_indices)
    if False:
        plot([go.Scatter(x=bcg_peak_indices[1:],y=bcg_intervals,mode="lines+markers",name="bcg"),
              go.Scatter(x=ecg_peak_indices[1:],
                         y=ecg_intervals,mode="lines+markers",name="ecg")],
             filename=bcg_file)
    break

/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/yjj_bcg_wave(500HZ)(03-15 145935).txt


/home/guo/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning:

Mean of empty slice.

/home/guo/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning:

invalid value encountered in double_scalars



In [10]:
from newtown import findpeaks_by_force

In [51]:
bcg = load_bcg(bcg_file, offset=39558-396)
bcg = bcg[0:500000]
ecg = ecg[0:500000]
diff = findpeaks_by_force(bcg)
if False:
    plot([go.Scatter(y=diff[1:],name="diff",marker=dict(color="green"),mode="lines+markers"),
          go.Scatter(y=np.abs(diff)[1:],name="abs(diff)",marker=dict(color="red"),mode="lines+markers"),
          go.Scatter(y=np.diff(diff),name="diff2",marker=dict(color="black"),mode="lines+markers"),
          go.Scatter(y=np.diff(np.abs(diff)),
                     name="np.diff(abs(diff))",marker=dict(color="yellow"),mode="lines+markers"),
          go.Scatter(y=np.asarray(bcg[2:])/10,name="bcg",marker=dict(color='blue'), mode="lines+markers"),
          go.Scatter(y=np.asarray(ecg[2:])/100,name="ecg",marker=dict(color="purple"))])

In [12]:
plot([go.Scatter(y=bcg,name="glt")])

'file:///home/guo/Github/BCGHeart/mean_hearts/temp-plot.html'

In [13]:
ecg

array([-1975.83831968, -1538.63651116,  -888.88376507, ...,
        -198.1925957 ,  -170.30674474,  -160.89834825])

# findpeaks2 dev

In [30]:
from findpeaks2 import findpeaks2

In [52]:
bcg = bcg[430000:450000]
ecg = ecg[430000:450000]

In [67]:
peaks, raw_peaks, thres_abs = findpeaks2(bcg,thres=0.65)
peak_indices = list(map(lambda peak:peak.peak_index, peaks))
peak_values = list(map(lambda peak:peak.peak_value/10, peaks))
raw_peak_indices = list(map(lambda peak:peak.peak_index, raw_peaks))
raw_indicator_value = list(map(lambda peak:peak.indicator_value, raw_peaks))


In [69]:
plot([go.Scatter(y=bcg/10,name="bcg"),
      go.Scatter(x=peak_indices, y=peak_values, mode="markers", name="peak"),
      go.Scatter(x=raw_peak_indices, y=raw_indicator_value, mode="lines+markers", name="indicator"),
      
      go.Scatter(y=ecg/100,name="ecg")])

'file:///home/guo/Github/BCGHeart/mean_hearts/temp-plot.html'

In [39]:
bcg

array([], dtype=float64)

In [49]:
ecg

array([], dtype=float64)

In [20]:
from bcg import load_bcg
import findpeaks2
import autopeaks2
import autopeaks
def findpeaks2_in_bcg(signal, direction="up"):
    bcg_auto_peaks = autopeaks2.FindPeaks(thres=0.2, min_dist=300,buffer_size=2000, direction=direction)
    list(map(bcg_auto_peaks.findpeaks, signal))
    bcg_peak_indices = bcg_auto_peaks.peak_indices
    bcg_peak_values = bcg_auto_peaks.peak_values
    raw_extremas = bcg_auto_peaks.raw_extremas
    raw_peaks = bcg_auto_peaks.raw_peaks
    thres_abs = bcg_auto_peaks.thres_abs
    extrema_index = list(map(lambda peak:peak.index, raw_extremas))
    extrema_value = list(map(lambda peak:peak.value, raw_extremas))
    raw_peaks_indices = list(map(lambda peak:peak.peak_index, raw_peaks))
    raw_peaks_scores = list(map(lambda peak:peak.peak_score, raw_peaks))
    raw_peaks_indicators = list(map(lambda peak:peak.indicator_value, raw_peaks))
    return bcg_peak_indices, bcg_peak_values, extrema_index, extrema_value, raw_peaks_indices, raw_peaks_indicators, raw_peaks_scores, thres_abs

def findpeaks_in_ecg(signal):
    ecg_auto_peaks = autopeaks.AutoPeaks(thres=0.7,min_dist=300,buffer_size=1000)
    list(map(ecg_auto_peaks.findpeaks, signal))
    return ecg_auto_peaks.peak_indexes, ecg_auto_peaks.peak_values

def findpeaks_for_bcg_and_ecg(bcg_file, ecg_file,direction="up",offset=0, is_return=False):
    if offset>=0:
        bcg_offset = offset
    else:
        bcg_offset = 0
    bcg = load_bcg(bcg_file, offset=bcg_offset)
    ecg = load_ecg(ecg_file, filter=True)
    if offset < 0:
        ecg = ecg[-offset:]
    #print(bcg[:100])
    start = 100000
    end = start + 400000
    #bcg = bcg[start:end]
    #ecg = ecg[start:end]
    bcg_peak_indices, bcg_peak_values, extrema_indices, extrema_values, raw_peaks_indices, raw_peaks_indicators, raw_peaks_scores, thres_abses =  findpeaks2_in_bcg(np.asarray(bcg), direction=direction)
    ecg_peak_indices, ecg_peak_values = findpeaks_in_ecg(ecg)
    #np.savetxt("glt.txt", np.diff(bcg_peak_indices[1:]), delimiter=',', fmt="%d")
    if is_return:
        return bcg_peak_indices, ecg_peak_indices
    if True:
        plot(go.Figure(data=[go.Scatter(x=bcg_peak_indices[1:],
                                        y=np.diff(bcg_peak_indices),  
                                        mode="lines+markers",
                                        name="bcg_intervals",
                                        marker=dict(color="blue")),
                             go.Scatter(x=ecg_peak_indices[1:], 
                                        y=np.diff(ecg_peak_indices), 
                                        mode="lines+markers", 
                                        name="ecg_intervals",
                                        marker=dict(color="green")),],                       
                            #go.Scatter(x=ecg_peak_indices[1:], 
                            #            y=np.diff(ecg_peak_indices)+30, 
                            #            mode="lines", 
                            #            name="ecg_intervals_upper",
                            #            marker=dict(color="red")),
                            #go.Scatter(x=ecg_peak_indices[1:], 
                            #            y=np.diff(ecg_peak_indices)-30, 
                            #            mode="lines", 
                            #            name="ecg_intervals",
                            #            marker=dict(color="red")),],
                      layout=go.Layout(yaxis=dict(range=[250,1500]))),filename=bcg_file+direction)
    if True:
        plot([go.Scatter(y=bcg[1:],name="bcg", marker=dict(color="blue")),
              #go.Scatter(y=np.diff(bcg), name="dff1", marker=dict(color="green")),
              #go.Scatter(y=np.abs(np.diff(bcg)), name="dff1_abs", marker=dict(color="red")),
              #go.Scatter(y=np.diff(np.abs(np.diff(bcg))), name="diff(dff1_abs)", marker=dict(color="yellow")),
              go.Scatter(x=bcg_peak_indices, y=np.asarray(bcg_peak_values), mode="markers", name="peak"),
              #go.Scatter(x=extrema_indices,y=extrema_values,
              #           mode="markers",
              #           name="raw_extremas",
              #           marker=dict(color="orange")),              
              #go.Scatter(x=raw_peaks_indices,y=raw_peaks_indicators,
              #           mode="lines+markers",
              #           name="raw_peaks_indicator",
              #           marker=dict(color="red")),
              #go.Scatter(x=raw_peaks_indices,y=raw_peaks_scores,
              #           mode="lines+markers",
              #           name="raw_peaks_score",
              #           marker=dict(color="deepskyblue")),
              #go.Scatter(y=thres_abses,
              #           mode="lines",
              #           name="thres_abs",
              #           marker=dict(color="black")),
              go.Scatter(y=ecg[1:]/20,name="ecg", marker=dict(color="purple"))],filename=bcg_file+direction)

In [32]:
from bcg import load_bcg
import findpeaks2
import autopeaks2
import autopeaks
def master_findpeaks2_in_bcg(signal):
    bcg_auto_peaks_master = autopeaks2.FindPeaksMaster(fs=500)
    list(map(bcg_auto_peaks_master.findpeaks, signal))
    bcg_peak_indices = bcg_auto_peaks_master.peak_indices
    bcg_peak_values = bcg_auto_peaks_master.peak_values
    bcg_peak_intervals = bcg_auto_peaks_master.peak_intervals
    #raw_extremas = bcg_auto_peaks.raw_extremas
    #raw_peaks = bcg_auto_peaks.raw_peaks
    #thres_abs = bcg_auto_peaks.thres_abs
    #extrema_index = list(map(lambda peak:peak.index, raw_extremas))
    #extrema_value = list(map(lambda peak:peak.value, raw_extremas))
    #raw_peaks_indices = list(map(lambda peak:peak.peak_index, raw_peaks))
    #raw_peaks_scores = list(map(lambda peak:peak.peak_score, raw_peaks))
    #raw_peaks_indicators = list(map(lambda peak:peak.indicator_value, raw_peaks))
    #return bcg_peak_indices, bcg_peak_values, extrema_index, extrema_value, raw_peaks_indices, raw_peaks_indicators, raw_peaks_scores, thres_abs
    return bcg_peak_indices, bcg_peak_intervals, bcg_peak_values

def findpeaks_in_ecg(signal):
    ecg_auto_peaks = autopeaks.AutoPeaks(thres=0.7,min_dist=300,buffer_size=1000)
    list(map(ecg_auto_peaks.findpeaks, signal))
    return ecg_auto_peaks.peak_indexes, ecg_auto_peaks.peak_values

def master_findpeaks_for_bcg_and_ecg(bcg_file, ecg_file, offset=0, is_return=False):
    if offset >= 0:
        bcg_offset = offset
    else:
        bcg_offset = 0
    bcg = load_bcg(bcg_file, offset=bcg_offset)
    ecg = load_ecg(ecg_file, filter=True)
    if offset < 0:
        ecg = ecg[-offset:]
    #print(bcg[:100])
    start = 100000
    end = start + 400000
    #bcg = bcg[start:end]
    #ecg = ecg[start:end]
    #bcg_peak_indices, bcg_peak_values, extrema_indices, extrema_values, raw_peaks_indices, raw_peaks_indicators, raw_peaks_scores, thres_abses =  findpeaks2_in_bcg(np.asarray(bcg), direction=direction)
    bcg_peak_indices, bcg_peak_intervals, bcg_peak_values =  master_findpeaks2_in_bcg(np.asarray(bcg))
    ecg_peak_indices, ecg_peak_values = findpeaks_in_ecg(ecg)
    #np.savetxt("glt.txt", np.diff(bcg_peak_indices[1:]), delimiter=',', fmt="%d")
    if is_return:
        return bcg_peak_indices, bcg_peak_intervals, ecg_peak_indices
    if False:
        plot(go.Figure(data=[go.Scatter(x=bcg_peak_indices[1:],
                                        y=np.diff(bcg_peak_indices),  
                                        mode="lines+markers",
                                        name="bcg_intervals",
                                        marker=dict(color="blue")),
                             go.Scatter(x=ecg_peak_indices[1:], 
                                        y=np.diff(ecg_peak_indices), 
                                        mode="lines+markers", 
                                        name="ecg_intervals",
                                        marker=dict(color="green")),],                       
                            #go.Scatter(x=ecg_peak_indices[1:], 
                            #            y=np.diff(ecg_peak_indices)+30, 
                            #            mode="lines", 
                            #            name="ecg_intervals_upper",
                            #            marker=dict(color="red")),
                            #go.Scatter(x=ecg_peak_indices[1:], 
                            #            y=np.diff(ecg_peak_indices)-30, 
                            #            mode="lines", 
                            #            name="ecg_intervals",
                            #            marker=dict(color="red")),],
                      layout=go.Layout(yaxis=dict(range=[250,1500]))),filename=bcg_file)
    if False:
        plot([go.Scatter(y=bcg[1:],name="bcg", marker=dict(color="blue")),
              #go.Scatter(y=np.diff(bcg), name="dff1", marker=dict(color="green")),
              #go.Scatter(y=np.abs(np.diff(bcg)), name="dff1_abs", marker=dict(color="red")),
              #go.Scatter(y=np.diff(np.abs(np.diff(bcg))), name="diff(dff1_abs)", marker=dict(color="yellow")),
              go.Scatter(x=bcg_peak_indices, y=np.asarray(bcg_peak_values), mode="markers", name="peak"),
              #go.Scatter(x=extrema_indices,y=extrema_values,
              #           mode="markers",
              #           name="raw_extremas",
              #           marker=dict(color="orange")),              
              #go.Scatter(x=raw_peaks_indices,y=raw_peaks_indicators,
              #           mode="lines+markers",
              #           name="raw_peaks_indicator",
              #           marker=dict(color="red")),
              #go.Scatter(x=raw_peaks_indices,y=raw_peaks_scores,
              #           mode="lines+markers",
              #           name="raw_peaks_score",
              #           marker=dict(color="deepskyblue")),
              #go.Scatter(y=thres_abses,
              #           mode="lines",
              #           name="thres_abs",
              #           marker=dict(color="black")),
              go.Scatter(y=ecg[1:]/20,name="ecg", marker=dict(color="purple"))],filename=bcg_file)

In [33]:
import time

def get_bcg_offset(bcg_file):
    offset = 0
    if "yjj" in bcg_file:
        offset = 39558-396
    elif "glt" in bcg_file:
        offset = 36621 + 39421 - 116
    elif "hwm" in bcg_file:
        offset = 1000 + 74953 - 169
    elif "lcc" in bcg_file:
        offset = 60000 + 8287 - 10
    elif "lifeng" in bcg_file:
        offset = 90000 - 25471 + 4396
    elif "zyh" in bcg_file:
        offset = 300000 - 14583
    elif "zcj" in bcg_file:
        offset = -10235 + 170
    elif "wkp" in bcg_file:
        offset = 5004
    return offset

for bcg_file,ecg_file in zip(bcg_files, ecg_files):
    #if "glt" not in bcg_file:
    #    continue
    print(bcg_file)
    offset = get_bcg_offset(bcg_file)
    print("offset: {}".format(offset))
    bcg_peak_indices, bcg_peak_intervals, ecg_peak_indices = master_findpeaks_for_bcg_and_ecg(bcg_file, ecg_file, offset=offset, is_return=True)
    #bcg_peak_indices_up, _ = findpeaks_for_bcg_and_ecg(bcg_file, ecg_file, offset=offset,direction="up", is_return=True)
    #time.sleep(1)
    #bcg_peak_indices_down, _ = findpeaks_for_bcg_and_ecg(bcg_file, ecg_file, offset=offset,direction="down", is_return=True)
    plot(go.Figure(data=[
        go.Scatter(x=bcg_peak_indices[1:],y=bcg_peak_intervals[1:], mode="lines+markers", marker=dict(color="black"),name="master"),
        #go.Scatter(x=bcg_peak_indices_up[1:],y=np.diff(bcg_peak_indices_up), mode="lines+markers", marker=dict(color="blue"), name="up"),
        #go.Scatter(x=bcg_peak_indices_down[1:],y=np.diff(bcg_peak_indices_down), mode="lines+markers", marker=dict(color="green"), name="down"),
        go.Scatter(x=ecg_peak_indices[1:],y=np.diff(ecg_peak_indices), mode="lines+markers", marker=dict(color="red"), name="ecg"),],layout=go.Layout(yaxis=dict(range=(300, 800)))),filename=bcg_file)

/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/glt_bcg_wave(500HZ)(03-15 142141).txt
offset: 75926


/home/guo/anaconda3/lib/python3.6/site-packages/plotly/offline/offline.py:451: UserWarning:

Your filename `/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/glt_bcg_wave(500HZ)(03-15 142141).txt` didn't end with .html. Adding .html to the end of your file.



/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/hwm_bcg_wave(500HZ)(03-15 200629).txt
offset: 75784


/home/guo/anaconda3/lib/python3.6/site-packages/plotly/offline/offline.py:451: UserWarning:

Your filename `/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/hwm_bcg_wave(500HZ)(03-15 200629).txt` didn't end with .html. Adding .html to the end of your file.



/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/lcc_bcg_wave(500HZ)(03-15 141708).txt
offset: 68277


...

/home/guo/anaconda3/lib/python3.6/site-packages/plotly/offline/offline.py:451: UserWarning:

Your filename `/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/yjj_bcg_wave(500HZ)(03-15 145935).txt` didn't end with .html. Adding .html to the end of your file.



/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/zcj_bcg_wave(500HZ)(03-15 154123).txt
offset: -10065


/home/guo/anaconda3/lib/python3.6/site-packages/plotly/offline/offline.py:451: UserWarning:

Your filename `/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/zcj_bcg_wave(500HZ)(03-15 154123).txt` didn't end with .html. Adding .html to the end of your file.



/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/zyh_bcg_wave(500HZ)(05-10 132226).txt
offset: 285417


/home/guo/anaconda3/lib/python3.6/site-packages/plotly/offline/offline.py:451: UserWarning:

Your filename `/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/zyh_bcg_wave(500HZ)(05-10 132226).txt` didn't end with .html. Adding .html to the end of your file.



In [ ]:
plot([go.Scatter(y=bcg/10,name="bcg"),
      go.Scatter(x=peak_indices, y=np.asarray(peak_values)/10, mode="markers", name="peak"),
      #go.Scatter(x=raw_peak_indices, y=raw_indicator_value, mode="lines+markers", name="indicator"),
      go.Scatter(y=ecg/100,name="ecg")])

In [45]:
import findpeaks2
from findpeaks2 import Peak

In [46]:
raw_peaks = [findpeaks2.Peak(peak_index=133, peak_value=171.47134869035574, front_index=122, front_value=9.3124041008777, front_front_index=111, front_front_value=9.08998513328347, back_index=144, back_value=5.907677284305521, back_back_index=161, back_back_value=7.588593216305682, indicator_value=15.22008138518322, peak_score=4), Peak(peak_index=321, peak_value=86.11892502506966, front_index=311, front_value=5.147337516434632, front_front_index=275, front_front_value=4.3090809159431736, back_index=336, back_value=10.348391835013182, back_back_index=368, back_back_value=5.563022101992402, indicator_value=15.495729351447814, peak_score=2), Peak(peak_index=413, peak_value=273.6858032668417, front_index=397, front_value=16.10281339447046, front_front_index=368, front_front_value=5.563022101992402, back_index=429, back_value=14.730496746553882, back_back_index=503, back_back_value=5.100019532254294, indicator_value=30.83331014102434, peak_score=6), Peak(peak_index=561, peak_value=128.23463671008494, front_index=537, front_value=5.632014670497085, front_front_index=503, front_front_value=5.100019532254294, back_index=576, back_value=6.827788521249431, back_back_index=597, back_back_value=4.744278060543429, indicator_value=12.459803191746516, peak_score=1), Peak(peak_index=740, peak_value=126.70827016420411, front_index=706, front_value=9.199566821369078, front_front_index=689, front_front_value=6.733857659132082, back_index=761, back_value=11.311799784682151, back_back_index=827, back_back_value=13.52210596002978, indicator_value=20.51136660605123, peak_score=4), Peak(peak_index=841, peak_value=142.01820873751709, front_index=827, front_value=13.52210596002978, front_front_index=761, front_front_value=11.311799784682151, back_index=857, back_value=10.702741211083797, back_back_index=885, back_back_value=2.849768954834076, indicator_value=24.224847171113577, peak_score=4)]

In [47]:
raw_peaks.sort(key=lambda peak:peak.peak_score)

In [48]:
raw_peaks

[Peak(peak_index=561, peak_value=128.23463671008494, front_index=537, front_value=5.632014670497085, front_front_index=503, front_front_value=5.100019532254294, back_index=576, back_value=6.827788521249431, back_back_index=597, back_back_value=4.744278060543429, indicator_value=12.459803191746516, peak_score=1),
 Peak(peak_index=321, peak_value=86.11892502506966, front_index=311, front_value=5.147337516434632, front_front_index=275, front_front_value=4.3090809159431736, back_index=336, back_value=10.348391835013182, back_back_index=368, back_back_value=5.563022101992402, indicator_value=15.495729351447814, peak_score=2),
 Peak(peak_index=133, peak_value=171.47134869035574, front_index=122, front_value=9.3124041008777, front_front_index=111, front_front_value=9.08998513328347, back_index=144, back_value=5.907677284305521, back_back_index=161, back_back_value=7.588593216305682, indicator_value=15.22008138518322, peak_score=4),
 Peak(peak_index=740, peak_value=126.70827016420411, front_in

In [49]:
peak_small = raw_peaks[-3]
peak_big = raw_peaks[-2]

In [50]:
peak_small > peak_big

False

In [51]:
peak_big > peak_small

True

In [52]:
peak_big,peak_small

(Peak(peak_index=841, peak_value=142.01820873751709, front_index=827, front_value=13.52210596002978, front_front_index=761, front_front_value=11.311799784682151, back_index=857, back_value=10.702741211083797, back_back_index=885, back_back_value=2.849768954834076, indicator_value=24.224847171113577, peak_score=4),
 Peak(peak_index=740, peak_value=126.70827016420411, front_index=706, front_value=9.199566821369078, front_front_index=689, front_front_value=6.733857659132082, back_index=761, back_value=11.311799784682151, back_back_index=827, back_back_value=13.52210596002978, indicator_value=20.51136660605123, peak_score=4))

In [79]:
peaks_with_score = []

def score_peak(curr_peak, peaks):
    score = 0
    for peak in peaks:
        if curr_peak > peak:
            score += 1
    return score

for curr_peak in raw_peaks:
    args = list(curr_peak)
    s = score_peak(curr_peak, raw_peaks)
    print(s)
    args[-1] = s
    peaks_with_score.append(Peak(*args))


0
1
3
3
3
5


In [80]:
peaks_with_score

[Peak(peak_index=561, peak_value=128.23463671008494, front_index=537, front_value=5.632014670497085, front_front_index=503, front_front_value=5.100019532254294, back_index=576, back_value=6.827788521249431, back_back_index=597, back_back_value=4.744278060543429, indicator_value=12.459803191746516, peak_score=0),
 Peak(peak_index=321, peak_value=86.11892502506966, front_index=311, front_value=5.147337516434632, front_front_index=275, front_front_value=4.3090809159431736, back_index=336, back_value=10.348391835013182, back_back_index=368, back_back_value=5.563022101992402, indicator_value=15.495729351447814, peak_score=1),
 Peak(peak_index=133, peak_value=171.47134869035574, front_index=122, front_value=9.3124041008777, front_front_index=111, front_front_value=9.08998513328347, back_index=144, back_value=5.907677284305521, back_back_index=161, back_back_value=7.588593216305682, indicator_value=15.22008138518322, peak_score=3),
 Peak(peak_index=740, peak_value=126.70827016420411, front_in

In [77]:
score_peak(peak_small,raw_peaks)

3

In [87]:
peak_big > raw_peaks[2]

False

In [62]:
args

[413,
 273.6858032668417,
 397,
 16.10281339447046,
 368,
 5.563022101992402,
 429,
 14.730496746553882,
 503,
 5.100019532254294,
 30.83331014102434,
 6]

In [90]:
np.asarray(raw_peaks,dtype=Peak)

array([[561, 128.23463671008494, 537, 5.632014670497085, 503,
        5.100019532254294, 576, 6.827788521249431, 597,
        4.744278060543429, 12.459803191746516, 1],
       [321, 86.11892502506966, 311, 5.147337516434632, 275,
        4.3090809159431736, 336, 10.348391835013182, 368,
        5.563022101992402, 15.495729351447814, 2],
       [133, 171.47134869035574, 122, 9.3124041008777, 111,
        9.08998513328347, 144, 5.907677284305521, 161, 7.588593216305682,
        15.22008138518322, 4],
       [740, 126.70827016420411, 706, 9.199566821369078, 689,
        6.733857659132082, 761, 11.311799784682151, 827,
        13.52210596002978, 20.51136660605123, 4],
       [841, 142.01820873751709, 827, 13.52210596002978, 761,
        11.311799784682151, 857, 10.702741211083797, 885,
        2.849768954834076, 24.224847171113577, 4],
       [413, 273.6858032668417, 397, 16.10281339447046, 368,
        5.563022101992402, 429, 14.730496746553882, 503,
        5.100019532254294, 30.83331014